In [0]:
%sql
SELECT 
    parent_award_agency_id,
    parent_award_agency_name,
    SUM(federal_action_obligation) AS total_obligation
FROM workspace.default.fact_contracts
GROUP BY parent_award_agency_id, parent_award_agency_name
ORDER BY total_obligation DESC
LIMIT 20;


In [0]:
%sql
SELECT 
    parent_award_agency_id,
    parent_award_agency_name,
    action_date,
    COUNT(*) AS num_transactions,
    SUM(federal_action_obligation) AS total_obligation
FROM workspace.default.fact_contracts
GROUP BY parent_award_agency_id, parent_award_agency_name, action_date
HAVING COUNT(*) > 1
   AND SUM(federal_action_obligation) > 100000
ORDER BY total_obligation DESC;


In [0]:
%sql
SELECT
    parent_award_agency_id,
    parent_award_agency_name,
    action_date,
    SUM(federal_action_obligation) OVER (
        PARTITION BY parent_award_agency_id
        ORDER BY action_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_obligation
FROM workspace.default.fact_contracts;


In [0]:
%sql
SELECT
    parent_award_agency_id,
    parent_award_agency_name,
    COUNT(CASE WHEN federal_action_obligation > 100000 THEN 1 END) AS large_awards,
    COUNT(*) AS total_awards
FROM workspace.default.fact_contracts
GROUP BY parent_award_agency_id, parent_award_agency_name;


In [0]:
%sql
SELECT 
    parent_award_agency_id,
    parent_award_agency_name,
    award_id_piid,
    action_date,
    federal_action_obligation
FROM workspace.default.fact_contracts
WHERE federal_action_obligation > 1000000
ORDER BY federal_action_obligation DESC
LIMIT 50;


In [0]:
%sql
WITH agency_daily AS (
    SELECT
        parent_award_agency_id,
        parent_award_agency_name,
        action_date,
        COUNT(*) AS num_transactions,
        SUM(federal_action_obligation) AS daily_total
    FROM workspace.default.fact_contracts
    GROUP BY parent_award_agency_id, parent_award_agency_name, action_date
)
SELECT *
FROM agency_daily
WHERE num_transactions > 1
  AND daily_total > 500000
ORDER BY daily_total DESC;

In [0]:
%sql
WITH agency_stats AS (
    SELECT
        parent_award_agency_id,
        parent_award_agency_name,
        PERCENTILE(federal_action_obligation, 0.99999) AS p99_obligation
    FROM workspace.default.fact_contracts
    GROUP BY parent_award_agency_id, parent_award_agency_name
)
SELECT f.*
FROM workspace.default.fact_contracts f
JOIN agency_stats s
  ON f.parent_award_agency_id = s.parent_award_agency_id
WHERE f.federal_action_obligation > s.p99_obligation
AND federal_action_obligation > 1000000
ORDER BY f.federal_action_obligation DESC;
